# Dataset Overview for ProphetNet tokenizer



In [1]:
#Install essential packages
%%capture
!pip install datasets
!pip install transformers

!apt install git-lfs

In [2]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers

In [3]:
from transformers import AutoTokenizer    
model_checkpoint = "microsoft/prophetnet-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/141 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

##XSum

In [4]:
# import dataset from cache
train_xsum = datasets.load_dataset("xsum", split="train")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [5]:
train_xsum

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

In [6]:
# Visualize the Data

import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    del df["id"]
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(train_xsum)

In [8]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["document"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["summary"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [9]:
# random choose 10% of the dataset
sample_size = 20000
data_stats = train_xsum.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (995 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (702 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1103 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


In [10]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 477.99315, %-Articles > 512:0.34115, Summary Mean:27.0488, %-Summary > 64:0.0017, %-Summary > 128:0.0


From the Result, we can see that for random 20000 sample (10% of training dataset), our articles has average token length of 478, and 34% is longer than 512. And the mean summary's token length is 27, almost none are longer than 64. 

It should be reasonable to use 512 for tokenized article length and 64 for tokenized summary length on ProphetNet tokenizer for Xsum dataset. But since Bart is limited to 1024, and to keep the same track with other dataset, we will stick to **512** and **64**.


##CNN/Daily

In [12]:
train_cnn = datasets.load_dataset("ccdv/cnn_dailymail", "3.0.0", split="train")

Downloading:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


In [13]:
train_cnn

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [14]:
show_random_elements(train_cnn)

,article,highlights
0,"Britain joined the U.S. and its allies in the fight against ISIS -- but any mission will come with a huge price tag. The UK parliament has approved airstrikes in Iraq against the terror group which prime minister David Cameron has called one of ""staggering"" brutality. Cameron had said Britain should join international allies in combating ISIS, an action that he warned would ""take not just months, but years."" While the total cost will depend on the intensity and length of operation, past experience can give some guidance, Malcolm Chalmers, research director from the Royal United Services Institute said. Chalmers pointed to the UK's seven month involvement in Libya through 2011, which cost around $390 million, including missiles and flight hours. The bill is additional to standard defense budgets, which include staffing costs and weapons spend. Last year, the UK defense budget was $60 billion. Chalmers said he would expect the cost of ISIS airstrikes to be in the ""ballpark,"" of the Libya operation, ""assuming we only use air power and the campaign lasts for a couple of years."" Cameron has said that the mission against ISIS could last ""years."" However, it is U.S.-led rather than the operation in Libya, in which European countries took the lead. 'Unique' missiles . Cameron, in Friday's debate aimed at convincing parliament to approve airstrikes, said Britain has ""unique assets"" no other coalition ally can contribute. Cameron named the Brimstone precision missile, which he said minimized risk of civilian casualties, as one only Britain could contribute. The missiles can target convoys as well as fast moving, fleeting targets that were previously harder to strike, according to IHS Jane's senior weapons analyst Ben Goodlad. The Brimstone, which travels at supersonic speed, is a 1.8-meter missile weighing just under 50 kg. Advanced radar equipment enables it to guides itself to the target, so pilots don't need to maneuver to release it. IHS Jane's estimates each missile costs around $240,000. Brimstone missiles are compatible with the UK's Tornado GR4 aircraft. The aircraft were used by British troops for relief missions in northern Iraq just over a month ago. Six of these are ready for deployment at the UK's RAF base in Cyprus. Flying the Tornadoes costs around $57,000 per hour, according to information released in a parliamentary debate about Libya in 2011. From the Cyprus base, the aircraft could reach Iraq in just over an hour. Goodlad said the RAF also has Tornado's Storm Shadow cruise missiles, which cost $1.3 million each, and submarine-launched Tomahawk cruise missiles. These can be used to target areas that can't be accessed by planes, for example due to defenses, and cost round $1.5 million each. U.S. forces have been using Tomahawk missiles for airstrikes in Syria earlier this week -- dropping 47 of them on Monday evening. In total, the U.S. has conducted 20 strikes in Syria and 198 in Iraq from August 8 through September 23. Many of those operations weren't included in the Pentagon's daily average spending figure of $7.5 million at the end of last month.","UK can start bombing ISIS within hours, as it has six fighter jets on standby in Cyprus .\nBritish forces have Brimstone precision missiles, high-tech weapons others don't have .\nThe U.S. is leading the airstrikes against ISIS in Iraq and Syria ."
1,"Japan has delayed the launch of its space probe for the second time due to bad weather. The Hayabusa2 will now blast off on a six-year mission to mine a distant asteroid aboard Japan's main H-IIA rocket from Tanegashima Space Center in south of the country on Wednesday at 1.22 pm local time. The probe was originally scheduled to launch today but Japan Aerospace Exploration Agency (JAXA) postponed it to Monday after a forecast of thick clouds. Hayabusa2, pictured, will now blast off from Tanegashima Space Center on Wednesday at 1.22 pm local time . Hayabusa2 had been set to blast off aboard 

In [15]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["article"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [16]:
sample_size = 25000
data_stats = train_cnn.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (1363 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1958 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (897 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1476 > 512). Running this sequence through the model will result in indexing errors


In [17]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 847.47, %-Articles > 512:0.73464, Summary Mean:56.77368, %-Summary > 64:0.29288, %-Summary > 128:0.0


We can see that on average an article contains 847 tokens with 3/4 of the articles being longer than 512. The summary is on average 57 tokens long. 30% our 25000-sample summaries are longer than 64 tokens, and almost none are longer than 128 tokens.

ProphetNet has limit of `max_length` 512 on tokenizer, It should be reasonable to use **512** for tokenized article length and **128** for tokenized summary length for CNN/Daily dataset.

##Wikihow

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!ls /content/drive/MyDrive/dataset

wikihowAll.csv


In [21]:
# import dataset from cache
dataset_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset")

Downloading:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Using custom data configuration all-6f5101161f12f62f


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikihow downloaded and prepared to /root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
train_wiki=dataset_wiki['train']
train_wiki

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 157252
})

In [23]:
# Visualize the Data

import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(train_wiki)

In [24]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["text"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["headline"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [25]:
sample_size = 20000
data_stats = train_wiki.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (714 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (785 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (751 > 512). Running this sequence through the model will result in indexing errors


In [26]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 741.6638, %-Articles > 512:0.5173, Summary Mean:66.92995, %-Summary > 64:0.38035, %-Summary > 128:0.1089


We can see that on average an article contains 742 tokens with half of the articles being longer than 512. The summary is on average 67 tokens long. 38% our 20000-sample summaries are longer than 64 tokens, and 10% are longer than 128 tokens.

ProphetNet has limit of `max_length` 512 on tokenizer, It should be reasonable to use **512** for tokenized article length and **128** for tokenized summary length for CNN/Daily dataset.